## SVM- restricted

In [1]:
import pandas as pd
import numpy as np
from sklearn import cluster, covariance, manifold
import numpy as np
from sklearn.metrics import r2_score
import math
import matplotlib.pyplot as plt
import time
import os

In [2]:
from sklearn.svm import SVR
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.optimize import minimize
from sklearn.feature_selection import RFECV

In [3]:
import optuna

In [4]:
# import warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error

In [6]:
path = r'C:\Users\User\Documents\Atividades_andamento\Nayana\corrige_codigos'
os.chdir(path)

In [7]:
#df =pd.read_excel('custos_versao_jun_17_2023.xlsx')
df =pd.read_excel("custos_versao_mar_14B_2024.xlsx")
df = df.drop(['Local','Barragens'],axis=1)
names = df.columns

In [8]:
df.shape

(33, 78)

In [9]:
word ='Custo_barragem' 
word = 'Dams'
X = df.drop([word],axis=1)
y = df[word]
y = np.log10(y)

escalor =  StandardScaler() # escalor = MinMaxScaler()
y_norm = escalor.fit_transform(y.values.reshape(-1,1))

## SVM



In [10]:
restricted =['Height', 'FencInst', 'RemovStruc', 'GeochAvai', 
             'GroundwaQMon', 'ResDep',  'SedDredg', 'ContMat']
#restricted = ['Volume', 'Height', 'Area', 'Ano de encerramento da operação da estrutura?']

In [11]:
X_restricted = df[restricted]

x_scaler = StandardScaler() # x_scaler = MinMaxScaler()
y_scaler = StandardScaler()# y_scaler = MinMaxScaler()

X_norm = x_scaler.fit_transform(X_restricted)
y_norm = y_scaler.fit_transform(y.values.reshape(-1,1))

In [12]:
X,y = X_norm,y_norm

In [13]:
def mine_test(set_parameters, X=X, y=y): # testa o ajuste
  rows=X.shape[0]
  cols =X.shape[1]

  y_pred   = []

  for i in range(rows):
    X_train = np.delete(X,i,axis=0)
    y_train = np.delete(y,i,axis=0)
    X_test  = X[i,:].reshape(1,cols)
    y_test  = y[i]
    # model =  XGBRegressor(**set_parameters)
    model =  SVR(**set_parameters)
    model = model.fit(X_train, y_train.reshape(-1,1))
    yi = model.predict(X_test)
    y_pred.append(yi)

  y_pred = np.array(y_pred)

  resp = mean_absolute_error(y, y_pred)
  

  return -resp


In [14]:
# Define the objective function and suggest hyperparameters values
def objective(trial):
    C_parameter = [100,10,1.0,0.10,0.01,0.001,0.0001,0.00001]
    eps_parameter = [0.01,0.1,1.0,2,10]
    #C = trial.suggest_float('C', 1e-10, 1e10, log=True)
    #gamma = trial.suggest_float('gamma', 1e-10, 1e10, log=True)
    #epsilon= trial.suggest_float('epsilon', 0.01, 1)
    epsilon = trial.suggest_categorical('epsilon', eps_parameter)
    C_fast = trial.suggest_categorical('C', C_parameter)
    # gamma nao tem sentido no kernel linear
    set_parameters = {'kernel':"linear",
                      'C': C_fast,
                      'epsilon':epsilon}
    adjust = mine_test( set_parameters)
    
    
    return adjust

In [15]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2024-08-27 10:46:24,297] A new study created in memory with name: no-name-b9a695db-0fed-4f55-ab2a-293909c98312
[I 2024-08-27 10:46:24,317] Trial 0 finished with value: -0.9188960359336699 and parameters: {'epsilon': 2, 'C': 0.01}. Best is trial 0 with value: -0.9188960359336699.
[I 2024-08-27 10:46:24,345] Trial 1 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:24,368] Trial 2 finished with value: -0.9188960359336699 and parameters: {'epsilon': 2, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:24,385] Trial 3 finished with value: -0.6226858013975085 and parameters: {'epsilon': 1.0, 'C': 100}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:24,409] Trial 4 finished with value: -0.9234397903030418 and parameters: {'epsilon': 10, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:24,429] Trial 5

[I 2024-08-27 10:46:26,694] Trial 47 finished with value: -0.9234397903030418 and parameters: {'epsilon': 10, 'C': 10}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:26,736] Trial 48 finished with value: -0.4654468639817672 and parameters: {'epsilon': 0.01, 'C': 1.0}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:26,759] Trial 49 finished with value: -0.8109417161034558 and parameters: {'epsilon': 0.1, 'C': 0.001}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:26,785] Trial 50 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:26,810] Trial 51 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:26,836] Trial 52 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 

[I 2024-08-27 10:46:29,815] Trial 95 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:29,827] Trial 96 finished with value: -0.9234397903030418 and parameters: {'epsilon': 10, 'C': 0.0001}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:29,866] Trial 97 finished with value: -0.41196170436851864 and parameters: {'epsilon': 0.01, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:29,876] Trial 98 finished with value: -0.9188960359336699 and parameters: {'epsilon': 2, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:29,917] Trial 99 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:29,943] Trial 100 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is tria

[I 2024-08-27 10:46:33,009] Trial 143 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:33,040] Trial 144 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:33,083] Trial 145 finished with value: -0.4634950718227744 and parameters: {'epsilon': 0.1, 'C': 1.0}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:33,112] Trial 146 finished with value: -0.9188960359336699 and parameters: {'epsilon': 2, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:33,141] Trial 147 finished with value: -0.8459797813638457 and parameters: {'epsilon': 0.1, 'C': 1e-05}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:33,169] Trial 148 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is t

[I 2024-08-27 10:46:35,447] Trial 191 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:35,477] Trial 192 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:35,498] Trial 193 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:35,533] Trial 194 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:35,562] Trial 195 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best is trial 1 with value: -0.40133314292721084.
[I 2024-08-27 10:46:35,589] Trial 196 finished with value: -0.40133314292721084 and parameters: {'epsilon': 0.1, 'C': 0.1}. Best i

In [16]:
# Print the best set of hyperparameters
print('Best hyperparameters: ', study.best_params)
# Print the corresponding performance
print('Best performance: ', study.best_value)

Best hyperparameters:  {'epsilon': 0.1, 'C': 0.1}
Best performance:  -0.40133314292721084


In [17]:
best = study.best_params

set_parameters = {'kernel':"linear",
                  'C': best['C'],
                  'epsilon':best['epsilon']}

In [18]:
def mine_test(X,y, set_parameters= set_parameters): # testa o ajuste
  rows=X.shape[0]
  cols =X.shape[1]

  y_pred   = []

  for i in range(rows):
    X_train = np.delete(X,i,axis=0)
    y_train = np.delete(y,i,axis=0)
    X_test  = X[i,:].reshape(1,cols)
    y_test  = y[i]
    model =  SVR(**set_parameters)
    model = model.fit(X_train, y_train.reshape(-1,1))
    yi = model.predict(X_test)
    y_pred.append(yi)

  y_pred = np.array(y_pred)

  return y_pred

In [19]:
y_pred = mine_test(X , y)
y_true = y

In [20]:
print('mae :', mean_absolute_error(y_true, y_pred))
print('rmse :', root_mean_squared_error(y_true, y_pred))
print('r-score :',r2_score(y_true, y_pred))

mae : 0.40133314292721084
rmse : 0.5280300554529093
r-score : 0.7211842605383976


In [21]:
for value in set_parameters:
    print(value,set_parameters[value])

kernel linear
C 0.1
epsilon 0.1
